## Module for building Henderson's Mixed Model Equations for single trait including marker  and polygenic effect



### <font color="red"> Data</font>

In [8]:
using DataFrames
using Distributions

In [9]:
FILE = "data/small.ped";

In [10]:
Animal = ["S1","D1","O1","O3"]
y = [-1.77, -1.40,0.89,-2.91]
df = DataFrame(Animal=Animal,y=y);

In [11]:
srand(123)
d = Binomial(2,0.5)
nObs     = 4
nMarkers = 5
M        = float(rand(d,(nObs,nMarkers)));

In [12]:
df=[df DataFrame(M)]

,Animal,y,x1,x2,x3,x4,x5
1,S1,-1.77,1.0,0.0,1.0,1.0,1.0
2,D1,-1.4,2.0,0.0,2.0,2.0,1.0
3,O1,0.89,1.0,2.0,0.0,1.0,0.0
4,O3,-2.91,0.0,0.0,2.0,1.0,1.0


### <font color="red"> Run module</font>

In [13]:
using MMEModule

In [14]:
varRes = 1.0
varGen = 2.5
markerProp = 0.8
MVarGen= varGen*markerProp
PVarGen= varGen*(1-markerProp);

In [15]:
ped = PedModule.mkPed(FILE);

In [16]:
mme = MMEModule.initMME("y = intercept + Animal",varRes);

In [17]:
G = PVarGen
MMEModule.setAsRandom(mme,"Animal",ped,G)

### <font color="blue"> Gauss-Seidel</font> 

In [18]:
resG = MMEModule.getSolG(mme,df,outFreq=100)

6x2 Array{Any,2}:
 "intercept: intercept"  -1.32835    
 "Animal: S1"            -0.0623349  
 "Animal: D1"             0.0607066  
 "Animal: O1"             0.443018   
 "Animal: O3"            -0.316982   
 "Animal: O2"            -0.000814176

### <font color="blue"> Gibbs</font> 

In [19]:
res2 = MMEModule.getSolGibbs(mme,df,outFreq=10000)

at sample: 10000
at sample: 20000
at sample: 30000
at sample: 40000
at sample: 50000


6x2 Array{Any,2}:
 "intercept: intercept"  -1.33148   
 "Animal: S1"            -0.0567149 
 "Animal: D1"             0.0639893 
 "Animal: O1"             0.448488  
 "Animal: O3"            -0.313325  
 "Animal: O2"             0.00484819

### <font color="red"> work with markers</font>

In [20]:
MMEModule.addMarkers(mme,df[:,3:7],MVarGen);

##### sample without vc

In [21]:
output=MMEModule.sampleMCMC(50000,mme,df,outFreq=10000)

at sample: 10000
at sample: 20000
at sample: 30000
at sample: 40000
at sample: 50000


Dict{Any,Any} with 2 entries:
  "posteriorMeanLocationP… => 6x2 Array{Any,2}:…
  "posteriorMeanMarkerEff… => [0.4014222266219835,0.617610218193303,-0.49622805…

In [22]:
output["posteriorMeanLocationParms"]

6x2 Array{Any,2}:
 "intercept: intercept"  -1.30033   
 "Animal: S1"            -0.0563279 
 "Animal: D1"             0.0573889 
 "Animal: O1"             0.088265  
 "Animal: O3"            -0.0832777 
 "Animal: O2"             0.00181328

In [23]:
output["posteriorMeanMarkerEffects"]

5-element Array{Float64,1}:
  0.401422
  0.61761 
 -0.496228
  0.107644
 -0.305358

##### sample vc

In [24]:
output=MMEModule.sampleMCMCv(500000,mme,df,outFreq=100000)

at sample: 100000 with meanVare: 0.7370977750701039 meanVara 0.6692961512286464 and G0Mean: [0.2825480899475662]
at sample: 200000 with meanVare: 0.7380709536814705 meanVara 0.6682686241708016 and G0Mean: [0.28272862480163224]
at sample: 300000 with meanVare: 0.7388636967298994 meanVara 0.6703262047708591 and G0Mean: [0.28262155944820666]
at sample: 400000 with meanVare: 0.7385118307575053 meanVara 0.6695506464814575 and G0Mean: [0.28248318325878097]
at sample: 500000 with meanVare: 0.7387515478289401 meanVara 0.6699670590017469 and G0Mean: [0.28279747111683917]


Dict{Any,Any} with 4 entries:
  "posteriorMeanLocationP… => 6x2 Array{Any,2}:…
  "posteriorMeanMarkerEff… => [0.4170068280996278,0.6217719439828568,-0.4849346…
  "MCMCSamples for geneti… => 500000x1 Array{Float64,2}:…
  "MCMCSamples for residu… => [0.241851,0.845866,0.671582,0.464386,0.0916222,0.…

In [25]:
keys(output)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "posteriorMeanLocationParms"
  "posteriorMeanMarkerEffects"
  "MCMCSamples for genetic var-cov parameters"
  "MCMCSamples for residual variance"

In [26]:
output["posteriorMeanLocationParms"]

6x2 Array{Any,2}:
 "intercept: intercept"  -1.29876    
 "Animal: S1"            -0.0274788  
 "Animal: D1"             0.0282235  
 "Animal: O1"             0.0408205  
 "Animal: O3"            -0.037919   
 "Animal: O2"            -0.000297438

In [27]:
output["posteriorMeanMarkerEffects"]

5-element Array{Float64,1}:
  0.417007
  0.621772
 -0.484935
  0.120861
 -0.309817